## 2c. Evidence - Performance QAS Measurements

Now we collect stored, CPU and memory usage data when predicting with the model, for the Performance scenario.

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from demo.scenarios.session import *

### Measurements

Prepare and execute measurements.

In [ ]:
from mlte.measurement.storage import LocalObjectSize
from mlte.evidence.types.integer import Integer

store_measurement = LocalObjectSize("model size")
size: Integer = store_measurement.evaluate(MODELS_DIR)
print(size)
size.save(force=True)

In [ ]:
from mlte.measurement.process_measurement import ProcessMeasurement
from mlte.measurement.cpu import LocalProcessCPUUtilization, CPUStatistics

cpu_measurement = LocalProcessCPUUtilization("predicting cpu")
cpu_stats: CPUStatistics = cpu_measurement.evaluate(
    ProcessMeasurement.start_script(MODEL_SCRIPT, MODEL_ARGS)
)
print(cpu_stats)
cpu_stats.save(force=True)

In [ ]:
from mlte.measurement.memory import (
    LocalProcessMemoryConsumption,
    MemoryStatistics,
)

mem_measurement = LocalProcessMemoryConsumption("predicting memory")
mem_stats: MemoryStatistics = mem_measurement.evaluate(
    ProcessMeasurement.start_script(MODEL_SCRIPT, MODEL_ARGS)
)
print(mem_stats)
mem_stats.save(force=True)